In [2]:
import pandas as pd 
import os 

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

import seaborn as sns

from datasets import load_dataset
import numpy as np

In [3]:
pwd

'/home/cn356/oliver/research/perturb-r/notebook'

In [5]:
cd /home/cn356/oliver/research/perturb-r

/home/cn356/oliver/research/perturb-r


In [6]:
from reward_score.math import * 

# Reasoning Results

In [20]:
root = "./results/hendrycks_math/sample200/reasoning"
os.listdir(root)

['R1-Distill-Qwen-1.5B.pickle',
 'R1-Distill-Qwen-1.5B_correct.pickle',
 'R1-Distill-Qwen-7B.pickle',
 'R1-Distill-Qwen-7B_correct.pickle',
 'Qwen2.5-Math-1.5B-DrGRPO-Zero.pickle',
 'Qwen2.5-Math-1.5B-DrGRPO-Zero_correct.pickle',
 'Qwen2.5-Math-7B-DrGRPO-Zero.pickle',
 'Qwen2.5-Math-7B-DrGRPO-Zero_correct.pickle',
 'Qwen2.5-Math-1.5B-DeepMath-Hard-GRPO-1800-steps.pickle',
 'Qwen2.5-Math-1.5B-DeepMath-Hard-GRPO-1800-steps_correct.pickle',
 'R1-Distill-Qwen-14B.pickle',
 'R1-Distill-Qwen-14B_correct.pickle',
 'DeepScaleR-1.5B-Preview.pickle',
 'DeepScaleR-1.5B-Preview_correct.pickle',
 'Qwen2.5-Math-1.5B-DeepMath-Hard-SFT-continue-3912-steps.pickle',
 'Qwen2.5-Math-1.5B-DeepMath-Hard-SFT-continue-3912-steps_correct.pickle']

In [21]:
for fname in os.listdir(root):
    if 'correct' not in fname:
        print(fname)
        df = pd.read_pickle(os.path.join(root, fname))
        print(f"Accuracy: {df['correct'].mean()}")

R1-Distill-Qwen-1.5B.pickle
Accuracy: 0.895
R1-Distill-Qwen-7B.pickle
Accuracy: 0.975
Qwen2.5-Math-1.5B-DrGRPO-Zero.pickle
Accuracy: 0.78
Qwen2.5-Math-7B-DrGRPO-Zero.pickle
Accuracy: 0.88
Qwen2.5-Math-1.5B-DeepMath-Hard-GRPO-1800-steps.pickle
Accuracy: 0.805
R1-Distill-Qwen-14B.pickle
Accuracy: 0.98
DeepScaleR-1.5B-Preview.pickle
Accuracy: 0.955
Qwen2.5-Math-1.5B-DeepMath-Hard-SFT-continue-3912-steps.pickle
Accuracy: 0.7


In [22]:
df = pd.read_pickle(os.path.join(root, "Qwen2.5-Math-1.5B-DeepMath-Hard-SFT-continue-3912-steps.pickle"))

In [26]:
df['response'][2]

"Okay, so I need to find a complex number z that satisfies the equation 3z + 4i times the conjugate of z equals 1 minus 8i. Hmm, complex numbers can be tricky, but maybe I can break this down into real and imaginary parts. Let me recall that a complex number z can be written as x + yi, where x and y are real numbers, and the conjugate of z, denoted as \\bar{z}, is x - yi. \n\nSo, let me start by letting z = x + yi. Then, the conjugate \\bar{z} would be x - yi. Now, substitute these into the equation given:\n\n3z + 4i \\bar{z} = 1 - 8i\n\nSubstituting z and \\bar{z}:\n\n3(x + yi) + 4i(x - yi) = 1 - 8i\n\nLet me expand each term step by step. First, expand 3z:\n\n3x + 3yi\n\nThen, expand 4i \\bar{z}:\n\n4i(x - yi) = 4i x - 4i^2 y\n\nSince i^2 is -1, this simplifies to:\n\n4i x - 4(-1)y = 4i x + 4y\n\nSo, combining both terms:\n\n3x + 3yi + 4y + 4i x = 1 - 8i\n\nNow, let's group the real parts and the imaginary parts together. The real parts are 3x and 4y, and the imaginary parts are 3y i

# Digit Corruption Results

In [12]:
def corruption_group_analysis (df: pd.DataFrame, percentile:str, stats: str):
    def group(pct):
        if pct <= .15:
            return .15
        elif pct <=.25:
            return .25
        elif pct <= 0.5:
            return .5
        elif pct <= .75:
            return .75
        else:
            return 1.0
    df['pctl_group'] = df[percentile].apply(group)
    return {"corruption percentage": df['pctl_group'].value_counts(), 
            "accuracy": df.groupby('pctl_group')[stats].mean()}

### Corrupt answer digit backwards

In [172]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-DrGRPO-Zero_type=answer_digit_continuethinking=False.pickle")

In [173]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 0.50    31
 1.00    27
 0.25    21
 0.75    19
 0.10    17
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.882353
 0.25    0.857143
 0.50    0.741935
 0.75    0.684211
 1.00    0.740741
 Name: still_correct, dtype: float64}

In [149]:
print(rl_df['post_corruption_response'][6])

 I started by recalling the definition of a complex number, which is \( z = x + yi \), where \( x \) and \( y \) are real numbers, and \( i \) is the imaginary unit. The conjugate of \( z \), denoted \( \bar{z} \), is \( x - yi \). 

I then substituted \( z = x + yi \) and \( \bar{z} = x - yi \) into the given equation \( 3z + 5i\bar{z} = 1 - 8i \). Expanding both terms, I got \( 3x + 3yi + 5ix - 5y = (3x + 5y) + (3y + 5x)i \). 

Setting the real and imaginary parts equal to the corresponding parts on the right-hand side, I obtained the system of linear equations: \( 3x + 5y = 1 \) and \( 5x + 3y = -8 \). 

To solve this system, I used the method of elimination. I multiplied the first equation by 5 and the second equation by 3 to align the coefficients of \( x \), resulting in the equations \( 12x + 16y = 5 \) and \( 12x + 9y = -25 \). Subtracting the second equation from the first, I found \( 7y = 28 \), leading to \( y = 5 \). Substituting \( y = 5 \) back into the first equation, I 

In [91]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/R1-Distill-Qwen-1.5B_type=answer_digit_continuethinking=False.pickle")

In [92]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 0.50    298
 0.25    290
 1.00    153
 0.75    151
 0.10    143
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.335664
 0.25    0.082759
 0.50    0.097315
 0.75    0.079470
 1.00    0.104575
 Name: still_correct, dtype: float64}

### Corrupt all digits forwards

In [174]:
sft_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-DrGRPO-Zero_type=midway_continuethinking=True.pickle")

In [175]:
corruption_group_analysis(sft_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 1.00    137
 0.50     71
 0.75     44
 0.25     27
 0.10      2
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.500000
 0.25    0.407407
 0.50    0.183099
 0.75    0.136364
 1.00    0.138686
 Name: still_correct, dtype: float64}

In [152]:
rl_df = pd.read_pickle("/share/goyal/lio/reasoning/eval/hendrycks_math/sample200/digit_corruption/Qwen2.5-Math-1.5B-sky-math8k-r1-continue-sft64steps-rl300steps_type=midway_continuethinking=True.pickle")

In [153]:
corruption_group_analysis(rl_df, 'percentile', 'still_correct')

{'corruption percentage': pctl_group
 1.00    106
 0.50     79
 0.75     65
 0.25     43
 0.10      4
 Name: count, dtype: int64,
 'accuracy': pctl_group
 0.10    0.250000
 0.25    0.302326
 0.50    0.113924
 0.75    0.107692
 1.00    0.216981
 Name: still_correct, dtype: float64}

# Distraction

## Full distraction

In [39]:
def if_answer (response):
    try:
        math_answer = remove_boxed(last_boxed_only_string(response))
        return True
    except:
        return False

In [8]:
root = './results/hendrycks_math/sample200/distraction'
os.listdir(root)

['Qwen2.5-Math-1.5B-DeepMath-Hard-SFT-continue-3912-steps.pickle',
 'Qwen2.5-Math-1.5B-DeepMath-Hard-GRPO-1800-steps.pickle',
 'DeepScaleR-1.5B-Preview.pickle',
 'Qwen2.5-Math-1.5B-DrGRPO-Zero.pickle',
 'Qwen2.5-Math-7B-DrGRPO-Zero.pickle',
 'R1-Distill-Qwen-1.5B.pickle',
 'R1-Distill-Qwen-7B.pickle']

In [57]:
root = './results/hendrycks_math/sample200/distraction'
df = pd.read_pickle(os.path.join(root, 'Qwen2.5-Math-1.5B-DeepMath-Hard-SFT-continue-3912-steps.pickle'))

df['if_answer'] = df['post_distraction_response'].apply(if_answer)
print("answer rate: ", df['if_answer'].mean())

print("accuracy according to original solution: \n", corruption_group_analysis(df, 'actual_fraction', 'original_correct'))
print('*' * 50, '\n\n')
print("accuracy according to distractor solution: \n", corruption_group_analysis(df, 'actual_fraction', 'distractor_correct'))

answer rate:  0.8821656050955414
accuracy according to original solution: 
 {'corruption percentage': pctl_group
0.25    130
0.50    130
0.75    130
1.00    130
0.15    108
Name: count, dtype: int64, 'accuracy': pctl_group
0.15    0.694444
0.25    0.707692
0.50    0.661538
0.75    0.584615
1.00    0.569231
Name: original_correct, dtype: object}
************************************************** 


accuracy according to distractor solution: 
 {'corruption percentage': pctl_group
0.25    130
0.50    130
0.75    130
1.00    130
0.15    108
Name: count, dtype: int64, 'accuracy': pctl_group
0.15    0.148148
0.25    0.123077
0.50    0.223077
0.75    0.230769
1.00    0.269231
Name: distractor_correct, dtype: object}


In [60]:
df['distractor_correct'].mean()

0.20063694267515925

In [64]:
df['post_distraction_response'].iloc[1]

'7 mod 8 is like 7, 15, 23, etc. So, the number of girls is 7 more than a multiple of 8.\n\nThen, if they line up in rows of four, how many are left over? The question is asking for the remainder when N is divided by 4. So, we need to find N mod 4. But since we know N mod 8, maybe we can relate these two moduli. Let\'s see.\n\nSince N ≡ 7 mod 8, then N can be written as 8k + 7. Let\'s divide that by 4. 8k divided by 4 is 2k, which is 0 mod 4. Then, 7 divided by 4 is 1 with a remainder of 3. So, 7 mod 4 is 3. Therefore, N mod 4 is (2k + 3) mod 4. But since 2k is a multiple of 2, adding 3 to it gives a number that is 3 mod 4. Wait, but hold on. Let me check that again.\n\nWait, if N = 8k + 7, then N divided by 4 is (8k)/4 + 7/4 = 2k + 1.75. But that\'s not helpful. Alternatively, maybe I can write N as 8k + 7. Let\'s compute N mod 4 directly. 8k mod 4 is 0, since 8 is divisible by 4. Then, 7 mod 4 is 3. Therefore, N mod 4 is 0 + 3 = 3. So, the remainder when N is divided by 4 is 3. There

In [71]:
root = './results/hendrycks_math/sample200/distraction'
df = pd.read_pickle(os.path.join(root, 'Qwen2.5-Math-1.5B-DeepMath-Hard-GRPO-1800-steps.pickle'))

df['if_answer'] = df['post_distraction_response'].apply(if_answer)
print("answer rate: ", df['if_answer'].mean())

print("accuracy according to original solution: \n", corruption_group_analysis(df, 'actual_fraction', 'original_correct'))
print('*' * 50, '\n\n')
print("accuracy according to distractor solution: \n", corruption_group_analysis(df, 'actual_fraction', 'distractor_correct'))

answer rate:  0.9676190476190476
accuracy according to original solution: 
 {'corruption percentage': pctl_group
0.50    151
0.75    151
0.25    110
1.00    101
0.15     12
Name: count, dtype: int64, 'accuracy': pctl_group
0.15    0.083333
0.25    0.281818
0.50    0.152318
0.75    0.125828
1.00     0.09901
Name: original_correct, dtype: object}
************************************************** 


accuracy according to distractor solution: 
 {'corruption percentage': pctl_group
0.50    151
0.75    151
0.25    110
1.00    101
0.15     12
Name: count, dtype: int64, 'accuracy': pctl_group
0.15    0.833333
0.25    0.372727
0.50    0.509934
0.75    0.503311
1.00    0.663366
Name: distractor_correct, dtype: object}


In [74]:
df['post_distraction_response'][0]

'1. **Identify the coordinates of the vertices:**\n   - \\(O(0,0)\\)\n   - \\(D(2,3)\\)\n   - \\(C(4,3)\\)\n   - \\(B(8,0)\\)\n\n2. **Determine the lengths of the parallel sides:**\n   - The length of \\(OC\\) (the top base) is the horizontal distance between \\(O\\) and \\(C\\), which is \\(4 - 0 = 4\\).\n   - The length of \\(BD\\) (the bottom base) is the horizontal distance between \\(B\\) and \\(D\\), which is \\(8 - 2 = 6\\).\n\n3. **Determine the height of the trapezoid:**\n   - The height is the vertical distance between the lines \\(OC\\) and \\(BD\\), which is \\(3 - 0 = 3\\).\n\n4. **Use the area formula for a trapezoid:**\n   The area \\(A\\) of a trapezoid is given by the formula:\n   \\[\n   A = \\frac{1}{2} \\times (\\text{sum of the lengths of the parallel sides}) \\times \\text{height}\n   \\]\n   Substituting the values we found:\n   \\[\n   A = \\frac{1}{2} \\times (4 + 6) \\times 3 = \\frac{1}{2} \\times 10 \\times 3 = 15\n   \\]\n\nTherefore, the area of trapezoid 

In [33]:
df

,problem,level,type,solution,original_response,problem_distractor,distract_solution,distract_reasoning,actual_fraction,post_distraction_response,original_correct,distractor_correct,pctl_group
0,<|im_start|>system\nPlease reason step by step...,Level 4,Prealgebra,The prime factorization of 48 is $2^4\cdot3$. ...,"Okay, so I need to find the other integer when...",<|im_start|>system\nPlease reason step by step...,"The number of girls is of the form $8n+7$, whe...","Okay, so here's this problem about Madeline's ...",0.125,"Wait, no, actually, the original problem is a...",1.0,False,0.25
1,<|im_start|>system\nPlease reason step by step...,Level 4,Prealgebra,The prime factorization of 48 is $2^4\cdot3$. ...,"Okay, so I need to find the other integer when...",<|im_start|>system\nPlease reason step by step...,"The number of girls is of the form $8n+7$, whe...","Okay, so here's this problem about Madeline's ...",0.250,"7 mod 8 is like 7, 15, 23, etc. So, the number...",False,False,0.25
2,<|im_start|>system\nPlease reason step by step...,Level 4,Prealgebra,The prime factorization of 48 is $2^4\cdot3$. ...,"Okay, so I need to find the other integer when...",<|im_start|>system\nPlease reason step by step...,"The number of girls is of the form $8n+7$, whe...","Okay, so here's this problem about Madeline's ...",0.500,3? But the problem mentions that when they lin...,True,False,0.50
3,<|im_start|>system\nPlease reason step by step...,Level 4,Prealgebra,The prime factorization of 48 is $2^4\cdot3$. ...,"Okay, so I need to find the other integer when...",<|im_start|>system\nPlease reason step by step...,"The number of girls is of the form $8n+7$, whe...","Okay, so here's this problem about Madeline's ...",0.750,"8k - 1 is 4*(2k - 1) + 3, right? Because 8k is...",1.0,False,0.75
4,<|im_start|>system\nPlease reason step by step...,Level 4,Prealgebra,The prime factorization of 48 is $2^4\cdot3$. ...,"Okay, so I need to find the other integer when...",<|im_start|>system\nPlease reason step by step...,"The number of girls is of the form $8n+7$, whe...","Okay, so here's this problem about Madeline's ...",0.875,24 divided by 8 is 3 with remainder 0. 24 divi...,False,False,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
645,<|im_start|>system\nPlease reason step by step...,Level 2,Algebra,We find the distance the two girls are to Robe...,"Okay, so I need to figure out how many units a...",<|im_start|>system\nPlease reason step by step...,Let $d$ be the common difference. We told tha...,"Okay, let's see. I need to find the first term...",0.125,"The nth term is a_n = a_1 + (n-1)d, where d i...",1.0,False,0.25
646,<|im_start|>system\nPlease reason step by step...,Level 2,Algebra,We find the distance the two girls are to Robe...,"Okay, so I need to figure out how many units a...",<|im_start|>system\nPlease reason step by step...,Let $d$ be the common difference. We told tha...,"Okay, let's see. I need to find the first term...",0.250,1st term equals third term minus the second te...,1.0,False,0.25
647,<|im_start|>system\nPlease reason step by step...,Level 2,Algebra,We find the distance the two girls are to Robe...,"Okay, so I need to figure out how many units a...",<|im_start|>system\nPlease reason step by step...,Let $d$ be the common difference. We told tha...,"Okay, let's see. I need to find the first term...",0.500,"2 = 6 - 4? 6 - 4 is 2, yes. So that checks out...",1.0,False,0.50
648,<|im_start|>system\nPlease reason step by step...,Level 2,Algebra,We find the distance the two girls are to Robe...,"Okay, so I need to figure out how many units a...",<|im_start|>system\nPlease reason step by step...,Let $d$ be the common difference. We told tha...,"Okay, let's see. I need to find the first term...",0.750,"2 is the first term, then 4, 6, 8. Yes, that s...",False,False,0.75


In [167]:
print(data['problem'])
print("+" * 50)
print(data['distract_reasoning'])
print("+" * 50)
print(data['post_distraction_response'])

<|im_start|>system
The user asks a qusetion and the assistant first thinks about the reasoning process and provides with the answer at the end. The reasoning process is enclosed within <think> </think> and put your final answer within \boxed{}.<|im_end|>
<|im_start|>user
What is the smallest positive multiple of $6$ that is also a multiple of $8$?<|im_end|>
<|im_start|>assistant
<think>
++++++++++++++++++++++++++++++++++++++++++++++++++
 So I need to find the value of \(a\) such that the function \(f(n)\) is continuous at \(n = a\). The function is defined as follows: \[f(n) = \begin{cases} 4n + 3 & \text{if } n < a \\ 7n - 12 & \text{if } n \ge a \end{cases}\] For the function to be continuous at \(n = a\), the left-hand limit as \(n\) approaches \(a\) must equal the right-hand limit as \(n\) approaches \(a\), and both must equal \(f(a)\). In other words, \[\lim_{n \to a^-} f(n) = \lim_{n \to a^+} f(n) = f(a)\] The left-hand limit is given by the expression for \(n < a\), which is \(4